In [1]:
import numpy as np
import pandas as pd
import optuna
import time
from numpy.linalg import inv, pinv
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.svm import SVC
from statsmodels.stats.proportion import proportion_confint
from sklearn.multiclass import OneVsOneClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score, cross_validate
from sklearn.utils.multiclass import unique_labels
from sklearn.utils import resample
from sklearn.base import BaseEstimator, ClassifierMixin
from typing import Optional
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')

New Dataset

In [2]:
train = pd.read_csv("/kaggle/input/intrusions/kdd_train.csv", sep=",")
test = pd.read_csv("/kaggle/input/intrusions/kdd_test.csv", sep=",")

In [3]:
data = pd.concat([train, test], ignore_index=True)
X = pd.get_dummies(data.iloc[:, :-1], columns=['protocol_type', 'service', 'flag'], dtype=int)
y = pd.get_dummies(data['labels'], dtype=int)

# paper says, they use 2000/4000/8000 random connections from dataset

# Extract the sampled records from X and Y datasets
sampled_X = X.sample(n=8000, random_state=42)
sampled_y = y.loc[sampled_X.index]

sampled_y_bin = (sampled_y['normal'] == 1).astype(int) # 1 is attack, 0 is normal
# train test split
#scaling

In [4]:
scaler = StandardScaler()

# then they split train and test equally
X_train = sampled_X[:4000]
onhs_tr = X_train.loc[:, "protocol_type_icmp":]
X_train = scaler.fit_transform(X_train.loc[:, :"dst_host_srv_rerror_rate"])
X_train = np.concatenate([X_train, onhs_tr.to_numpy()], axis=1)

X_test = sampled_X[4000:]
onhs_te = X_test.loc[:, "protocol_type_icmp":]
X_test = scaler.fit_transform(X_test.loc[:, :"dst_host_srv_rerror_rate"])
X_test = np.concatenate([X_test, onhs_te.to_numpy()], axis=1)

y_train = sampled_y[:4000]
y_train_bin = sampled_y_bin[:4000]
y_train = np.argmax(y_train, axis=1)

y_test = sampled_y[4000:]
y_test_bin = sampled_y_bin[4000:]
y_test = np.argmax(y_test, axis=1)

In [5]:
class ELM(BaseEstimator, ClassifierMixin):
    def __init__(self, num_input_nodes : int=None, num_hidden_units : int=None,
                 num_out_units : int=None, C : float=None,
                 beta_init : np.ndarray = None,
                 w_init : np.ndarray = None,
                 bias_init : np.ndarray = None):
        self.num_input_nodes = num_input_nodes
        self.num_hidden_units = num_hidden_units
        self.num_out_units = num_out_units
        self.C = C

        if isinstance(beta_init, np.ndarray):
            self._beta = beta_init
        else:
            self._beta = np.random.uniform(-1., 1., size=(self.num_hidden_units, self.num_out_units))

        if isinstance(w_init, np.ndarray):
            self._w = w_init
        else:
            self._w = np.random.uniform(-1, 1, size=(self.num_input_nodes, self.num_hidden_units))

        if isinstance(bias_init, np.ndarray):
            self._bias = bias_init
        else:
            self._bias = np.zeros(shape=(self.num_hidden_units,))

    def _sigmoid(self, x : float) ->  float:
        return 1. / (1. + np.exp(-x))

    def fit(self, X : np.ndarray, Y : np.ndarray) -> None:
        self.classes_ = unique_labels(Y)
        
        H = self._sigmoid(X.dot(self._w) + self._bias)
        I = np.eye(self.num_hidden_units)
        
        label_mapping = {cls: i for i, cls in enumerate(self.classes_)}
        Y_mapped = np.array([label_mapping[label] for label in Y])

        # Convert labels to one-hot encoding
        Y_onehot = np.zeros((Y.shape[0], self.classes_.shape[0]))
        Y_onehot[np.arange(Y.shape[0]), Y_mapped] = 1
        
        self._beta = np.linalg.inv((H.T @ H) + (self.C * I)) @ H.T @ Y_onehot


    def predict_proba(self, X : np.ndarray) -> np.ndarray:
        H = self._sigmoid(X.dot(self._w) + self._bias)
        preds = H.dot(self._beta)
        
        if self.num_out_units == 1:
        # For binary classification, use sigmoid function
            y_pred_proba = 1 / (1 + np.exp(-preds))
        else:
        # For multi-class classification, use softmax function
            exp_pred = np.exp(preds - np.max(preds, axis=1, keepdims=True))
            y_pred_proba = exp_pred / np.sum(exp_pred, axis=1, keepdims=True)

        return y_pred_proba
    
    def predict(self, X : np.ndarray) -> np.ndarray:
        y_pred_proba = self.predict_proba(X)
        y_pred_indices = np.argmax(y_pred_proba, axis=1)
        y_pred = self.classes_[y_pred_indices]
        return y_pred
    
    def classification_report(self, X : np.ndarray, y_true : np.ndarray) -> str:
        y_pred = self.predict(X)
        return classification_report(y_true, y_pred)
    
    def get_params(self, deep=True):
        return {
            'num_out_units': self.num_out_units,
            'num_hidden_units': self.num_hidden_units,
            'num_out_units': self.num_out_units,
            'C': self.C
        }

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

In [6]:
class KELM(BaseEstimator, ClassifierMixin):
    def __init__(self, num_out_units : int=None, C : float=None, gamma : float=None):
        self.num_out_units = num_out_units
        self.C = C
        self.gamma = gamma
        self._beta = None

    def _cal_K(self, X : np.ndarray, Y : np.ndarray) -> np.ndarray:
        if self.gamma is None:
            self.gamma = 1.0 / X.shape[1]  # Default gamma value
        # Compute the RBF kernel matrix
        X_norm = np.sum(X ** 2, axis=-1)
        Y_norm = np.sum(Y ** 2, axis=-1)
        K = np.exp(-self.gamma * (X_norm[:, None] + Y_norm[None, :] - 2 * np.dot(X, Y.T)))
        return K
    
    def fit(self, X : np.ndarray, Y : np.ndarray) -> None:
        self.classes_ = unique_labels(Y)
        self._X_train = X
        if self.C is None: # Default C value
            self.C = 1.0
        K = self._cal_K(X, X)  # Kernel matrix for training data
        reg_term = np.eye(K.shape[0]) / self.C

        # Map labels to contiguous integers starting from 0
        label_mapping = {cls: i for i, cls in enumerate(self.classes_)}
        Y_mapped = np.array([label_mapping[label] for label in Y])

        # Convert labels to one-hot encoding
        Y_onehot = np.zeros((Y.shape[0], self.classes_.shape[0]))
        Y_onehot[np.arange(Y.shape[0]), Y_mapped] = 1

        # Compute the output weights beta
        self._beta = np.linalg.solve(K + reg_term, Y_onehot)
        #self._beta = np.linalg.solve(K + reg_term, Y)

    def predict_proba(self, X : np.ndarray) -> np.ndarray:
        K = self._cal_K(X, self._X_train)  # Kernel matrix between new data and training data
        preds = np.dot(K, self._beta)
        
        if self.num_out_units == 1:
        # For binary classification, use sigmoid function
            y_pred_proba = 1 / (1 + np.exp(-preds))
        else:
        # For multi-class classification, use softmax function
            exp_pred = np.exp(preds - np.max(preds, axis=1, keepdims=True))
            y_pred_proba = exp_pred / np.sum(exp_pred, axis=1, keepdims=True)

        return y_pred_proba
    
    def predict(self, X : np.ndarray) -> np.ndarray:
        y_pred_proba = self.predict_proba(X)
        y_pred_indices = np.argmax(y_pred_proba, axis=1)
        y_pred = self.classes_[y_pred_indices]
        return y_pred
    
    def classification_report(self, X : np.ndarray, y_true : np.ndarray) -> str:
        y_pred = self.predict(X)
        return classification_report(y_true, y_pred)
    
    def get_params(self, deep=True):
        return {
            'num_out_units': self.num_out_units,
            'C': self.C,
            'gamma': self.gamma
        }

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

In [7]:
def objective_KELM_bin(trial):
    # Define the hyperparameter search space
    C = trial.suggest_float('C', 2**-24, 2**25, log=True)
    gamma = trial.suggest_float('gamma', 2**-24, 2**25, log=True)
    num_out_units = trial.suggest_categorical('num_out_units', [2])

    # Create the KELM model with the sampled hyperparameters
    model = KELM(num_out_units=num_out_units, C=C, gamma=gamma)

    # Perform cross-validation and compute the mean accuracy score
    scores = cross_val_score(model, X_train, y_train_bin, cv=10, n_jobs=-1)
    mean_score = scores.mean()

    return mean_score

In [8]:
def objective_KELM(trial):
    # Define the hyperparameter search space
    C = trial.suggest_float('C', 2**-24, 2**25, log=True)
    gamma = trial.suggest_float('gamma', 2**-24, 2**25, log=True)
    num_out_units = trial.suggest_categorical('num_out_units', [37])

    # Create the KELM model with the sampled hyperparameters
    model = KELM(num_out_units=num_out_units, C=C, gamma=gamma)

    # Perform cross-validation and compute the mean accuracy score
    scores = cross_val_score(model, X_train, y_train, cv=10, n_jobs=-1)
    mean_score = scores.mean()

    return mean_score

In [9]:
def objective_SVC_bin(trial):
    # Define the hyperparameter search space
    C = trial.suggest_float('C', 2**-24, 2**25, log=True)
    gamma = trial.suggest_float('gamma', 2**-24, 2**25, log=True)
    kernel = trial.suggest_categorical('kernel', ['rbf'])

    # Create the KELM model with the sampled hyperparameters
    model = SVC(C=C, gamma=gamma, kernel=kernel)

    # Perform cross-validation and compute the mean accuracy score
    scores = cross_val_score(model, X_train, y_train_bin, cv=10, n_jobs=-1)
    mean_score = scores.mean()

    return mean_score

In [10]:
def objective_SVC(trial):
    # Define the hyperparameter search space
    C = trial.suggest_float('C', 2**-24, 2**25, log=True)
    gamma = trial.suggest_float('gamma', 2**-24, 2**25, log=True)
    kernel = trial.suggest_categorical('kernel', ['rbf'])

    # Create the KELM model with the sampled hyperparameters
    model = SVC(C=C, gamma=gamma, kernel=kernel)

    # Perform cross-validation and compute the mean accuracy score
    scores = cross_val_score(model, X_train, y_train, cv=10, n_jobs=-1)
    mean_score = scores.mean()

    return mean_score

In [11]:
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [12]:
# KELM BINARY
# Create an Optuna study
study = optuna.create_study(direction='maximize')

# Optimize the hyperparameters
study.optimize(objective_KELM_bin, n_trials=200)

# Get the best parameters and best score
best_params_KELM_bin = study.best_params
best_score = study.best_value

print("Best params: ", best_params_KELM_bin)
print("Best score: ", best_score)

# Visualize the optimization history
optuna.visualization.plot_optimization_history(study)

# Visualize the parameter importances
optuna.visualization.plot_param_importances(study)

# Create accuracy plots for each hyperparameter
hyperparameters = ['C', 'gamma']
for param in hyperparameters:
    # Visualize the parameter relationship with accuracy
    fig = optuna.visualization.plot_slice(study, params=[param])
    fig.show()

# Best params:  {'C': 2695.319524333166, 'gamma': 0.01553735916384662, 'num_out_units': 2}
# Best score:  0.9874999999999998
# Best params:  {'C': 48121.41012465776, 'gamma': 0.003090353150154412, 'num_out_units': 2}
# Best score:  0.9872500000000001

Best params:  {'C': 9364.66046045252, 'gamma': 0.007474629215043808, 'num_out_units': 2}
Best score:  0.9880000000000001


In [13]:
# KELM
# Create an Optuna study
study = optuna.create_study(direction='maximize')

# Optimize the hyperparameters
study.optimize(objective_KELM, n_trials=200)

# Get the best parameters and best score
best_params_KELM = study.best_params
best_score = study.best_value

print("Best params: ", best_params_KELM)
print("Best score: ", best_score)

# Visualize the optimization history
optuna.visualization.plot_optimization_history(study)

# Visualize the parameter importances
optuna.visualization.plot_param_importances(study)

# Create accuracy plots for each hyperparameter
hyperparameters = ['C', 'gamma']
for param in hyperparameters:
    # Visualize the parameter relationship with accuracy
    fig = optuna.visualization.plot_slice(study, params=[param])
    fig.show()

# Best params:  {'C': 693.4338672757635, 'gamma': 0.037970452844140454, 'num_out_units': 37}
# Best score:  0.9860000000000001
# Best params:  {'C': 2149.624455236542, 'gamma': 0.02036344623502429, 'num_out_units': 37}
# Best score:  0.9855

Best params:  {'C': 698.8458989849286, 'gamma': 0.035461439652010165, 'num_out_units': 37}
Best score:  0.9862500000000001


In [14]:
# SVC BINARY
# Create an Optuna study
study = optuna.create_study(direction='maximize')

# Optimize the hyperparameters
study.optimize(objective_SVC_bin, n_trials=200)

# Get the best parameters and best score
best_params_SVC_bin = study.best_params
best_score = study.best_value

print("Best params: ", best_params_SVC_bin)
print("Best score: ", best_score)

# Visualize the optimization history
optuna.visualization.plot_optimization_history(study)

# Visualize the parameter importances
optuna.visualization.plot_param_importances(study)

# Create accuracy plots for each hyperparameter
hyperparameters = ['C', 'gamma']
for param in hyperparameters:
    # Visualize the parameter relationship with accuracy
    fig = optuna.visualization.plot_slice(study, params=[param])
    fig.show()

# Best params:  {'C': 59.5191479489651, 'gamma': 0.031963463688337294, 'kernel': 'rbf'}
# Best score:  0.9865
# Best params:  {'C': 2.651715530823638, 'gamma': 0.2143432904111394, 'kernel': 'rbf'}
# Best score:  0.98575

Best params:  {'C': 391.14684625382284, 'gamma': 0.02916532074296789, 'kernel': 'rbf'}
Best score:  0.9865


In [15]:
# SVC
# Create an Optuna study
study = optuna.create_study(direction='maximize')

# Optimize the hyperparameters
study.optimize(objective_SVC, n_trials=200)

# Get the best parameters and best score
best_params_SVC = study.best_params
best_score = study.best_value

print("Best params: ", best_params_SVC)
print("Best score: ", best_score)

# Visualize the optimization history
optuna.visualization.plot_optimization_history(study)

# Visualize the parameter importances
optuna.visualization.plot_param_importances(study)

# Create accuracy plots for each hyperparameter
hyperparameters = ['C', 'gamma']
for param in hyperparameters:
    # Visualize the parameter relationship with accuracy
    fig = optuna.visualization.plot_slice(study, params=[param])
    fig.show()

# Best params:  {'C': 126.20701596403441, 'gamma': 0.012668493747853154, 'kernel': 'rbf'}
# Best score:  0.9845
# Best params:  {'C': 129.19717801945768, 'gamma': 0.013344723038063278, 'kernel': 'rbf'}
# Best score:  0.9845

Best params:  {'C': 145.7669011724858, 'gamma': 0.021273885895855472, 'kernel': 'rbf'}
Best score:  0.9852500000000001


Testing tuned classifiers

In [16]:
classifiers = {
    # num_hidden_units=400 and C=2 for ELM proposed by paper
    "ELM for binary": ELM(122, 400, 2, C=2), 
    "ELM for multiclass": ELM(122, 400, 37, C=2),
    "Kernel ELM for binary": KELM(**best_params_KELM_bin),
    "Kernel ELM for multiclass": KELM(**best_params_KELM),
    "SVM for binary": SVC(**best_params_SVC_bin),
    "SVM for multiclass": SVC(**best_params_SVC)
}

In [17]:
for name, classifier in classifiers.items():
    # Fit the classifier on the training data
    if name.endswith("binary"):
        ys_train = y_train_bin
        ys_test = y_test_bin
    else:
        ys_train = y_train
        ys_test = y_test
    if name.startswith("ELM"):
        # because ELM initializes wit
        times = []
        accuracies = []
    start_time = time.time()
    classifier.fit(X_train, ys_train)
    end_time = time.time()
    fit_time = end_time - start_time
    
    # Make predictions on the test data
    start_time = time.time()
    y_pred = classifier.predict(X_test)
    end_time = time.time()
    predict_time = end_time - start_time
    
    # Calculate the accuracy
    accuracy = accuracy_score(ys_test, y_pred)
    
    # Calculate the 95% confidence interval
    n_samples = len(ys_test)
    lower, upper = proportion_confint(int(accuracy * n_samples), n_samples, alpha=0.05, method='wilson')
    
    print(f"Classifier: {name}")
    print(f"Fit Time: {fit_time:.4f} seconds")
    print(f"Predict Time: {predict_time:.4f} seconds")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"95% Confidence Interval: [{lower:.4f}, {upper:.4f}]")
    print()

Classifier: ELM for binary
Fit Time: 0.1732 seconds
Predict Time: 0.0656 seconds
Accuracy: 0.9858
95% Confidence Interval: [0.9816, 0.9890]

Classifier: ELM for multiclass
Fit Time: 0.1094 seconds
Predict Time: 0.0656 seconds
Accuracy: 0.9782
95% Confidence Interval: [0.9732, 0.9823]

Classifier: Kernel ELM for binary
Fit Time: 1.9819 seconds
Predict Time: 0.4411 seconds
Accuracy: 0.9855
95% Confidence Interval: [0.9813, 0.9888]

Classifier: Kernel ELM for multiclass
Fit Time: 1.7862 seconds
Predict Time: 0.4691 seconds
Accuracy: 0.9842
95% Confidence Interval: [0.9799, 0.9877]

Classifier: SVM for binary
Fit Time: 0.2069 seconds
Predict Time: 0.1193 seconds
Accuracy: 0.9865
95% Confidence Interval: [0.9824, 0.9896]

Classifier: SVM for multiclass
Fit Time: 0.2826 seconds
Predict Time: 0.3026 seconds
Accuracy: 0.9870
95% Confidence Interval: [0.9830, 0.9901]

